## Working with Transformers in the HuggingFace Ecosystem

In this laboratory exercise we will learn how to work with the HuggingFace ecosystem to adapt models to new tasks. As you will see, much of what is required is *investigation* into the inner-workings of the HuggingFace abstractions. With a little work, a little trial-and-error, it is fairly easy to get a working adaptation pipeline up and running.

### Exercise 1: Sentiment Analysis (warm up)

In this first exercise we will start from a pre-trained BERT transformer and build up a model able to perform text sentiment analysis. Transformers are complex beasts, so we will build up our pipeline in several explorative and incremental steps.

#### Exercise 1.1: Dataset Splits and Pre-trained model
There are a many sentiment analysis datasets, but we will use one of the smallest ones available: the [Cornell Rotten Tomatoes movie review dataset](cornell-movie-review-data/rotten_tomatoes), which consists of 5,331 positive and 5,331 negative processed sentences from the Rotten Tomatoes movie reviews.

**Your first task**: Load the dataset and figure out what splits are available and how to get them. Spend some time exploring the dataset to see how it is organized. Note that we will be using the [HuggingFace Datasets](https://huggingface.co/docs/datasets/en/index) library for downloading, accessing, splitting, and batching data for training and evaluation.

In [1]:
from datasets import load_dataset

dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes")
print(dataset)

#load separate splits
train_dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes", split="train")
valid_dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes", split="validation")
test_dataset  = load_dataset("cornell-movie-review-data/rotten_tomatoes", split="test")

f:\UniversitE\Deep Learning Application Lab\Transformer\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})


In [2]:
train_dataset['text'][0:5], train_dataset['label'][0:5]

(['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .',
  'effective but too-tepid biopic',
  'if you sometimes like to go to the movies to have fun , wasabi is a good place to start .',
  "emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one ."],
 [1, 1, 1, 1, 1])

In [3]:
print("Train:", "Positive", train_dataset['label'].count(1), "Negative", train_dataset['label'].count(0))
print("Validation:", "Positive", valid_dataset['label'].count(1), "Negative",  valid_dataset['label'].count(0))
print("Test:", "Positive", test_dataset['label'].count(1), "Negative",  test_dataset['label'].count(0))

Train: Positive 4265 Negative 4265
Validation: Positive 533 Negative 533
Test: Positive 533 Negative 533


#### Exercise 1.2: A Pre-trained BERT and Tokenizer

The model we will use is a *very* small BERT transformer called [Distilbert](https://huggingface.co/distilbert/distilbert-base-uncased) this model was trained (using self-supervised learning) on the same corpus as BERT but using the full BERT base model as a *teacher*.

**Your next task**: Load the Distilbert model and corresponding tokenizer. Use the tokenizer on a few samples from the dataset and pass the tokens through the model to see what outputs are provided. I suggest you use the [`AutoModel`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html) class (and the `from_pretrained()` method) to load the model and `AutoTokenizer` to load the tokenizer).

In [4]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")

In [5]:
token = tokenizer(train_dataset['text'][0:5], padding=True, truncation=True, return_tensors="pt")
token['input_ids'][0][0:10], token['attention_mask'][0][0:10] #token content (first 10 token/attention mask of first review)
token['input_ids'].shape, token['attention_mask'].shape # number of sentence, sequence length

(torch.Size([5, 52]), torch.Size([5, 52]))

In [6]:
model_output = model(token.input_ids, attention_mask=token.attention_mask)
print(model_output.last_hidden_state.shape) # number of sentence, toker for sentence, hidden size

torch.Size([5, 52, 768])


In [7]:
model_output.last_hidden_state[0].shape

torch.Size([52, 768])

#### Exercise 1.3: A Stable Baseline

In this exercise I want you to:
1. Use Distilbert as a *feature extractor* to extract representations of the text strings from the dataset splits;
2. Train a classifier (your choice, by an SVM from Scikit-learn is an easy choice).
3. Evaluate performance on the validation and test splits.

These results are our *stable baseline* -- the **starting** point on which we will (hopefully) improve in the next exercise.

**Hint**: There are a number of ways to implement the feature extractor, but probably the best is to use a [feature extraction `pipeline`](https://huggingface.co/tasks/feature-extraction). You will need to interpret the output of the pipeline and extract only the `[CLS]` token from the *last* transformer layer. *How can you figure out which output that is?*

In [8]:
from sklearn import svm
from transformers import pipeline

feature_extractor = pipeline("feature-extraction", framework="pt", model="distilbert-base-uncased", tokenizer="distilbert-base-uncased")

features_train = feature_extractor(train_dataset['text'], return_tensors="pt")
features_validation = feature_extractor(valid_dataset['text'], return_tensors="pt")
features_test = feature_extractor(test_dataset['text'], return_tensors="pt")

Device set to use cuda:0


In [9]:
import numpy as np
#extract CLS token first one from the paper
CLS_token_train = np.array([feature[0][0] for feature in features_train])
labels_train = train_dataset['label']
CLS_token_validation = np.array([feature[0][0] for feature in features_validation])
labels_validation = valid_dataset['label']
CLS_token_test = np.array([feature[0][0] for feature in features_test])
labels_test = test_dataset['label']


In [10]:
print(CLS_token_train.shape, len(labels_train))
print(CLS_token_validation.shape, len(labels_validation))
print(CLS_token_test.shape, len(labels_test))

# Binary classification only positive and negative sentences
set(labels_train), set(labels_validation), set(labels_test)

(8530, 768) 8530
(1066, 768) 1066
(1066, 768) 1066


({0, 1}, {0, 1}, {0, 1})

In [11]:
svm_model = svm.SVC(random_state=42) # seed
svm_model.fit(CLS_token_train, labels_train)

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [69]:
svm_pred_validation = svm_model.predict(CLS_token_validation)
svm_pred_test = svm_model.predict(CLS_token_test)

In [74]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_final_metrics_svm(predictions, labels):
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
    


In [75]:
print("Validation: ", compute_final_metrics_svm(svm_pred_validation, labels_validation))
print("Test: ",compute_final_metrics_svm(svm_pred_test, labels_test))

Validation:  {'accuracy': 0.8142589118198874, 'f1': 0.8043478260869565, 'precision': 0.8496868475991649, 'recall': 0.7636022514071295}
Test:  {'accuracy': 0.7945590994371482, 'f1': 0.7855044074436827, 'precision': 0.8217213114754098, 'recall': 0.7523452157598499}


Baseline

Validation:  {'accuracy': 0.8142589118198874, 'f1': 0.8043478260869565, 'precision': 0.8496868475991649, 'recall': 0.7636022514071295}

Test:  {'accuracy': 0.7945590994371482, 'f1': 0.7855044074436827, 'precision': 0.8217213114754098, 'recall': 0.7523452157598499}

-----
### Exercise 2: Fine-tuning Distilbert

In this exercise we will fine-tune the Distilbert model to (hopefully) improve sentiment analysis performance.

#### Exercise 2.1: Token Preprocessing

The first thing we need to do is *tokenize* our dataset splits. Our current datasets return a dictionary with *strings*, but we want *input token ids* (i.e. the output of the tokenizer). This is easy enough to do my hand, but the HugginFace `Dataset` class provides convenient, efficient, and *lazy* methods. See the documentation for [`Dataset.map`](https://huggingface.co/docs/datasets/v3.5.0/en/package_reference/main_classes#datasets.Dataset.map).

**Tip**: Verify that your new datasets are returning for every element: `text`, `label`, `intput_ids`, and `attention_mask`.

In [13]:
def tokenizer(dataset):
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
    tokens = tokenizer(dataset['text'], padding=True, return_tensors="pt")
    return tokens

# Tokenize the dataset
train_token = train_dataset.map(tokenizer, batched=True)
validation_token = valid_dataset.map(tokenizer, batched=True)
test_token = test_dataset.map(tokenizer, batched=True)

In [14]:
train_token, validation_token, test_token

(Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 8530
 }),
 Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 1066
 }),
 Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 1066
 }))

#### Exercise 2.2: Setting up the Model to be Fine-tuned

In this exercise we need to prepare the base Distilbert model for fine-tuning for a *sequence classification task*. This means, at the very least, appending a new, randomly-initialized classification head connected to the `[CLS]` token of the last transformer layer. Luckily, HuggingFace already provides an `AutoModel` for just this type of instantiation: [`AutoModelForSequenceClassification`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelforsequenceclassification). You will want you instantiate one of these for fine-tuning.

In [49]:
from transformers import AutoModelForSequenceClassification
classification_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
#model structure
classification_model.to('cuda')
classification_model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [16]:
print("base model parameters:", model.num_parameters())
print("classification model parameters", classification_model.num_parameters())

base model parameters: 66362880
classification model parameters 66955010


#### Exercise 2.3: Fine-tuning Distilbert

Finally. In this exercise you should use a HuggingFace [`Trainer`](https://huggingface.co/docs/transformers/main/en/trainer) to fine-tune your model on the Rotten Tomatoes training split. Setting up the trainer will involve (at least):


1. Instantiating a [`DataCollatorWithPadding`](https://huggingface.co/docs/transformers/en/main_classes/data_collator) object which is what *actually* does your batch construction (by padding all sequences to the same length).
2. Writing an *evaluation function* that will measure the classification accuracy. This function takes a single argument which is a tuple containing `(logits, labels)` which you should use to compute classification accuracy (and maybe other metrics like F1 score, precision, recall) and return a `dict` with these metrics.  
3. Instantiating a [`TrainingArguments`](https://huggingface.co/docs/transformers/v4.51.1/en/main_classes/trainer#transformers.TrainingArguments) object using some reasonable defaults.
4. Instantiating a `Trainer` object using your train and validation splits, you data collator, and function to compute performance metrics.
5. Calling `trainer.train()`, waiting, waiting some more, and then calling `trainer.evaluate()` to see how it did.

**Tip**: When prototyping this laboratory I discovered the HuggingFace [Evaluate library](https://huggingface.co/docs/evaluate/en/index) which provides evaluation metrics. However I found it to have insufferable layers of abstraction and getting actual metrics computed. I suggest just using the Scikit-learn metrics...

In [73]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_final_metrics(logits, labels):
    predictions = logits.argmax(axis=-1)
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=AutoTokenizer.from_pretrained("distilbert-base-uncased"), return_tensors="pt")

training_args = TrainingArguments(
    output_dir = "./Output/rotten_tomatoes",
    eval_strategy = "epoch",
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    optim = "sgd",
    weight_decay = 0.01,
    learning_rate = 0.001,
    num_train_epochs = 100,
    save_strategy = "best",
    seed = 104,
    metric_for_best_model = "eval_loss"
)
trainer = Trainer(
    model=classification_model,
    args=training_args,
    train_dataset=train_token,
    eval_dataset=validation_token,
    data_collator=data_collator
)

In [56]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.343100,0.371391
2,0.340500,0.378080
3,0.342300,0.374193
4,0.335000,0.377633
5,0.336300,0.369336
6,0.337000,0.368293
7,0.328700,0.371984
8,0.332400,0.372273
9,0.330600,0.371518
10,0.335600,0.372258


TrainOutput(global_step=26700, training_loss=0.32191091273161354, metrics={'train_runtime': 798.6924, 'train_samples_per_second': 533.998, 'train_steps_per_second': 33.43, 'total_flos': 8525863090142040.0, 'train_loss': 0.32191091273161354, 'epoch': 50.0})

In [58]:
#Save model
trainer.save_model("./Output/rotten_tomatoes/after_100_epochs")

In [ ]:
#load wanted checkpoint
trainer._load_from_checkpoint("./Output/rotten_tomatoes/best_model")
# trainer._load_from_checkpoint("./Output/rotten_tomatoes/after_100_epochs")

In [77]:
print(trainer.evaluate())
prediction_validation = trainer.predict(validation_token)
metrics_validation = compute_final_metrics(prediction_validation.predictions, prediction_validation.label_ids)
print("Validation Metrics:", metrics_validation)
prediction_test = trainer.predict(test_token)
metrics_test = compute_final_metrics(prediction_test.predictions, prediction_test.label_ids)
print("Test Metrics:", metrics_test)

{'eval_loss': 0.3742266297340393, 'eval_runtime': 0.6589, 'eval_samples_per_second': 1617.753, 'eval_steps_per_second': 101.679, 'epoch': 50.0}
Validation Metrics: {'accuracy': 0.8320825515947468, 'f1': 0.8344125809435707, 'precision': 0.822992700729927, 'recall': 0.8461538461538461}


Test Metrics: {'accuracy': 0.8283302063789869, 'f1': 0.8278457196613358, 'precision': 0.8301886792452831, 'recall': 0.8255159474671669}


## Best Model

eval_loss: 0.3712210953235626

Validation Metrics: {'accuracy': 0.8358348968105066, 'f1': 0.8341232227488151, 'precision': 0.842911877394636, 'recall': 0.8255159474671669}

Test Metrics: {'accuracy': 0.8320825515947468, 'f1': 0.829686013320647, 'precision': 0.8416988416988417, 'recall': 0.8180112570356473}

## Baseline

Validation:  {'accuracy': 0.8142589118198874, 'f1': 0.8043478260869565, 'precision': 0.8496868475991649, 'recall': 0.7636022514071295}

Test:  {'accuracy': 0.7945590994371482, 'f1': 0.7855044074436827, 'precision': 0.8217213114754098, 'recall': 0.7523452157598499}

-----
### Exercise 3: Choose at Least One


#### Exercise 3.2: Fine-tuning a CLIP Model (harder) (Made in separate file)

Use a (small) CLIP model like [`openai/clip-vit-base-patch16`](https://huggingface.co/openai/clip-vit-base-patch16) and evaluate its zero-shot performance on a small image classification dataset like ImageNette or TinyImageNet. Fine-tune (using a parameter-efficient method!) the CLIP model to see how much improvement you can squeeze out of it.

**Note**: There are several ways to adapt the CLIP model; you could fine-tune the image encoder, the text encoder, or both. Or, you could experiment with prompt learning.

**Tip**: CLIP probably already works very well on ImageNet and ImageNet-like images. For extra fun, look for an image classification dataset with different image types (e.g. *sketches*).